In [1]:
import re
import time
import requests
from bs4 import BeautifulSoup

import pandas as pd

In [5]:
import requests
from bs4 import BeautifulSoup, NavigableString
import re
import pandas as pd
import os

# 저장 경로
os.makedirs('./data', exist_ok=True)
output_path = './data/recipe_single_final.csv'

# 대상 URL
recipe_url = "https://www.10000recipe.com/recipe/7039774"
headers = {'User-Agent': 'Mozilla/5.0'}

try:
    response = requests.get(recipe_url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # ✅ 제목
    title_tag = soup.select_one('#relationGoods > div.best_tit > b:nth-child(1)')
    title = title_tag.text.strip() if title_tag else None
    print("제목:", title)

    # ✅ 인분
    servings_tag = soup.select_one('.view2_summary_info1')
    servings = re.sub(r'[^0-9]', '', servings_tag.text) if servings_tag else None
    print("인분:", servings)

    # ✅ 조리 시간
    time_tag = soup.select_one('.view2_summary_info2')
    cooking_time = time_tag.text.strip() if time_tag else None
    print("조리 시간:", cooking_time)

    # ✅ 재료
    ingredients = []
    ingredient_sections = soup.select('#divConfirmedMaterialArea > ul')
    for ul in ingredient_sections:
        for li in ul.select('li'):
            name_tag = li.select_one('.ingre_list_name')
            amount_tag = li.select_one('.ingre_list_ea')

            name = name_tag.text.strip() if name_tag else ''
            amount = amount_tag.text.strip() if amount_tag else ''

            name = re.sub(r'\s+', ' ', name)
            amount = re.sub(r'\s+', ' ', amount)

            if name:
                ingredients.append(f"{name} {amount}".strip())

    print("재료:", ingredients)

    # ✅ 조리 순서 (하위 태그 제거)
    cooking_order = []
    for i in range(1, 30):
        step_tag = soup.select_one(f'#stepdescr{i}')
        if not step_tag:
            break

        # 하위 태그 제외, 텍스트 노드만 추출
        step_text = ''.join([str(content).strip() for content in step_tag.contents if isinstance(content, NavigableString)])
        step_text = re.sub(r'\s+', ' ', step_text).strip()

        if step_text:
            cooking_order.append(step_text)

    print("조리순서:", cooking_order)

    # ✅ DataFrame 생성
    df = pd.DataFrame([{
        '제목': title,
        'URL': recipe_url,
        '인분': servings,
        '조리시간': cooking_time,
        '재료': '|'.join(ingredients),
        '조리순서': '|'.join(cooking_order)
    }])

    # ✅ 저장
    df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"\n✅ 저장 완료: {output_path}\n")

    # ✅ 결과 확인
    print("📄 저장된 데이터:")
    print(df.head(1).to_markdown(index=False))

except Exception as e:
    print("❗ 예외 발생:", e)


제목: 차돌박이떡볶음
인분: 2
조리 시간: 20분 이내
재료: ['떡국떡 300g', '차돌박이 200g', '새송이버섯 2개', '대파 1대', '청양고추 1개', '진간장 3숟가락', '설탕 2숟가락', '굴소스 2숟가락']
조리순서: ['떡국떡은 물에 10분 정도 불린다.', '차돌박이를 준비한다.', '새송이버섯은 2~4등분으로 잘라 삼각형 모양으로 자른다.', '대파는 어슷 썬다.', '청양고추는 어슷 썬다.', '불린 떡국떡은 물기를 제거한다.', '팬에 기름 없이 차돌박이를 센 불로 볶는다.', '차돌박이가 절반 정도 익으면 차돌박이는 건져낸다.', '차돌박이 기름에 떡국떡, 야채를 넣고 중불로 볶는다.', '야채가 절반 익으면 새송이버섯, 진간장, 설탕, 굴소스를 넣고 센 불로 볶는다.', '모든 재료가 익으면 차돌박이를 넣고 센 불로 볶는다.']

✅ 저장 완료: ./data/recipe_single_final.csv

📄 저장된 데이터:
| 제목           | URL                                        |   인분 | 조리시간   | 재료                                                                                                      | 조리순서                                                                                                                                                                                                                                                                                                                                      

# 여러개 저장 리스트에 넣기

In [8]:
import requests
from bs4 import BeautifulSoup, NavigableString
import re
import pandas as pd
import os

# 저장 경로
os.makedirs('./data', exist_ok=True)
output_path = './data/recipe_multi_final.csv'

# ✅ 레시피 ID 목록 (숫자만!)
recipe_ids = [6878062, 6946621, 6837942, 6921799]

# ✅ 기본 URL
base_url = "https://www.10000recipe.com/recipe/"

# ✅ headers
headers = {'User-Agent': 'Mozilla/5.0'}

# ✅ 결과 저장 리스트
all_recipes = []

# ✅ 반복 수집
for recipe_id in recipe_ids:
    recipe_url = f"{base_url}{recipe_id}"

    try:
        response = requests.get(recipe_url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        # ✅ 제목
        title_tag = soup.select_one('#relationGoods > div.best_tit > b:nth-child(1)')
        title = title_tag.text.strip() if title_tag else None

        # ✅ 인분
        servings_tag = soup.select_one('.view2_summary_info1')
        servings = re.sub(r'[^0-9]', '', servings_tag.text) if servings_tag else None

        # ✅ 조리 시간
        time_tag = soup.select_one('.view2_summary_info2')
        cooking_time = time_tag.text.strip() if time_tag else None

        # ✅ 재료
        ingredients = []
        ingredient_sections = soup.select('#divConfirmedMaterialArea > ul')
        for ul in ingredient_sections:
            for li in ul.select('li'):
                name_tag = li.select_one('.ingre_list_name')
                amount_tag = li.select_one('.ingre_list_ea')

                name = name_tag.text.strip() if name_tag else ''
                amount = amount_tag.text.strip() if amount_tag else ''

                name = re.sub(r'\s+', ' ', name)
                amount = re.sub(r'\s+', ' ', amount)

                if name:
                    ingredients.append(f"{name} {amount}".strip())

        # ✅ 조리 순서 (하위 태그 제외)
        cooking_order = []
        for i in range(1, 30):
            step_tag = soup.select_one(f'#stepdescr{i}')
            if not step_tag:
                break

            step_text = ''.join([str(content).strip() for content in step_tag.contents if isinstance(content, NavigableString)])
            step_text = re.sub(r'\s+', ' ', step_text).strip()

            if step_text:
                cooking_order.append(step_text)

        # ✅ 결과 저장
        all_recipes.append({
            '레시피ID': recipe_id,
            '제목': title,
            'URL': recipe_url,
            '인분': servings,
            '조리시간': cooking_time,
            '재료': '|'.join(ingredients),
            '조리순서': '|'.join(cooking_order)
        })

        print(f"✅ 수집 완료: {title} (ID: {recipe_id})")

    except Exception as e:
        print(f"❌ 예외 발생 (ID: {recipe_id}):", e)

# ✅ 저장
df = pd.DataFrame(all_recipes)
df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"\n📄 총 {len(df)}개 레시피 저장 완료: {output_path}")
print(df.head())


✅ 수집 완료: 순대국밥 (ID: 6878062)
✅ 수집 완료: 순대국밥 (ID: 6946621)
✅ 수집 완료: 김치콩나물국밥 (ID: 6837942)
✅ 수집 완료: 노른자기장밥 (ID: 6921799)

📄 총 4개 레시피 저장 완료: ./data/recipe_multi_final.csv
     레시피ID       제목                                         URL 인분    조리시간  \
0  6878062     순대국밥  https://www.10000recipe.com/recipe/6878062  4  30분 이내   
1  6946621     순대국밥  https://www.10000recipe.com/recipe/6946621  2  15분 이내   
2  6837942  김치콩나물국밥  https://www.10000recipe.com/recipe/6837942  2  30분 이내   
3  6921799   노른자기장밥  https://www.10000recipe.com/recipe/6921799  1  60분 이내   

                                                  재료  \
0  시판 순대 4컵|대파 1대|청양고추 2개|시판 곰탕 600g 1봉지|물 2컵|소금 ...   
1  일반 순대 1인분|사골곰탕 1팩|물 500g|깻잎 10장|대파 2개|다진 마늘 1스...   
2  신김치 조금|콩나물 2줌|멸치다시마육수 6컵|계란 1개|다진파 약간|청양고추 1개|...   
3  기장 밥 50g|감자 1/2개|삶은 달걀노른자 1개|브로콜리 15g|당근 5g|캐슈...   

                                                조리순서  
0  그릇에 양념장을 만들어주세요. 다진 마늘 1.5, 들깨가루 2.5, 고춧가루 3,새...  
1  1. 육수만들기 :사골육수 1팩 (500g)을 넣어주고 같은 양의 물(500g)을 ...

In [9]:
df = pd.read_csv(output_path)
df.head()

,레시피ID,제목,URL,인분,조리시간,재료,조리순서
0,6878062,순대국밥,https://www.10000recipe.com/recipe/6878062,4,30분 이내,시판 순대 4컵|대파 1대|청양고추 2개|시판 곰탕 600g 1봉지|물 2컵|소금 ...,"그릇에 양념장을 만들어주세요. 다진 마늘 1.5, 들깨가루 2.5, 고춧가루 3,새..."
1,6946621,순대국밥,https://www.10000recipe.com/recipe/6946621,2,15분 이내,일반 순대 1인분|사골곰탕 1팩|물 500g|깻잎 10장|대파 2개|다진 마늘 1스...,1. 육수만들기 :사골육수 1팩 (500g)을 넣어주고 같은 양의 물(500g)을 ...
2,6837942,김치콩나물국밥,https://www.10000recipe.com/recipe/6837942,2,30분 이내,신김치 조금|콩나물 2줌|멸치다시마육수 6컵|계란 1개|다진파 약간|청양고추 1개|...,콩나물은 깨끗이 씻어 준비하고|김치는 송송 썰어 주세요.|냄비에 콩나물과 송송썬 김...
3,6921799,노른자기장밥,https://www.10000recipe.com/recipe/6921799,1,60분 이내,기장 밥 50g|감자 1/2개|삶은 달걀노른자 1개|브로콜리 15g|당근 5g|캐슈...,"쌀, 기장 5:1 비율로 밥을 짓는다.|믹서에 두유, 브로콜리, 당근, 볶은 캐슈너..."
